In [2]:
import pandas as pd

# Read CSV
df = pd.read_csv("/Crop_Production - Crop_Production.csv")

# Columns with leading/trailing spaces
cols_with_spaces = [col for col in df.columns if col.strip() != col]
print("Columns with extra spaces:", cols_with_spaces)

# Example: check inconsistent spellings or capitalization in 'District'
print(df['District'].unique())


Columns with extra spaces: []
['NICOBARS' 'NORTH AND MIDDLE ANDAMAN' 'SOUTH ANDAMANS' 'ANANTAPUR'
 'EAST GODAVARI' 'KRISHNA' 'VIZIANAGARAM' 'WEST GODAVARI' 'ADILABAD'
 'CHITTOOR' 'GUNTUR' 'KADAPA' 'KARIMNAGAR' 'KHAMMAM' 'KURNOOL'
 'MAHBUBNAGAR' 'MEDAK' 'NALGONDA' 'NIZAMABAD' 'PRAKASAM' 'RANGAREDDI'
 'SPSR NELLORE' 'SRIKAKULAM' 'VISAKHAPATANAM' 'WARANGAL' 'HYDERABAD'
 'DIBANG VALLEY' 'EAST KAMENG' 'EAST SIANG' 'KAMLE' 'LEPARADA' 'LOHIT'
 'LONGDING' 'LOWER DIBANG VALLEY' 'LOWER SIANG' 'LOWER SUBANSIRI' 'NAMSAI'
 'PAKKE KESSANG' 'PAPUM PARE' 'SIANG' 'TIRAP' 'UPPER SIANG'
 'UPPER SUBANSIRI' 'WEST SIANG' 'ANJAW' 'CHANGLANG' 'KRA DAADI'
 'KURUNG KUMEY' 'SHI YOMI' 'TAWANG' 'WEST KAMENG' 'BAKSA' 'BARPETA'
 'BISWANATH\n' 'BONGAIGAON' 'CACHAR' 'CHARAIDEO\n' 'CHIRANG' 'DARRANG'
 'DHEMAJI' 'DHUBRI' 'DIBRUGARH' 'DIMA HASAO' 'GOALPARA' 'GOLAGHAT'
 'HAILAKANDI' 'HOJAI\n' 'JORHAT' 'KAMRUP' 'KAMRUP METRO' 'KARBI ANGLONG'
 'KARIMGANJ' 'KOKRAJHAR' 'LAKHIMPUR' 'MAJULI\n' 'MARIGAON' 'NAGAON'
 'NALBARI' 'SI

In [3]:
top5_yield = (
    df.groupby('Crop')['Yield']
      .mean()
      .sort_values(ascending=False)
      .head(5)
)
print(top5_yield)


Crop
Coconut      8833.247721
Sugarcane      55.592646
Banana         29.052143
Tapioca        18.905141
Onion          13.178732
Name: Yield, dtype: float64


In [4]:
rare_crops = df['Crop'].value_counts()
rare_crops = rare_crops[rare_crops < 300]
print(rare_crops)


Crop
Other Summer Pulses    67
Name: count, dtype: int64


In [5]:
df.rename(columns=lambda x: x.strip(), inplace=True)


In [6]:
seasonal_perf = df.groupby(['Season', 'Crop']).agg({
    'Production': 'sum',
    'Yield': 'mean'
}).reset_index()

print(seasonal_perf.head())


   Season          Crop  Production      Yield
0  Autumn      Arecanut      6180.0   0.745000
1  Autumn     Arhar/Tur      6487.0   0.202778
2  Autumn        Banana     31605.0  11.917500
3  Autumn  Black pepper        66.0   0.055000
4  Autumn  Cotton(lint)        20.0   0.670000


In [8]:
yearwise_prod = df.groupby(['Crop', 'Crop_Year'])['Production'].mean().unstack()
print(yearwise_prod.head())

Crop_Year          1997          1998          1999          2000  \
Crop                                                                
Arecanut    4114.453333   9867.580247   2643.629032   4038.806818   
Arhar/Tur   4100.397129   6266.597619   6155.463768   5013.069042   
Bajra      28403.517241  23245.348684  17817.461538  19217.538889   
Banana     37492.697674  49691.468254  40553.167832  39178.508876   
Barley      7119.686957   6117.012097   6584.465116   5413.839695   

Crop_Year          2001          2002          2003          2004  \
Crop                                                                
Arecanut    4278.568182   3045.163934   4485.795699   5317.625000   
Arhar/Tur   4978.984479   4763.400531   4775.293634   4986.037838   
Bajra      23397.045198  13315.704225  32816.671196  22913.119318   
Banana     35143.358333  33564.615789  48056.600877  44147.727273   
Barley      5454.007692   5323.729730   4890.408240   4652.480159   

Crop_Year          2005         

In [11]:
avg_yield_year = df.groupby(['Crop', 'Crop_Year'])['Yield'].mean().reset_index()
highest_yield = avg_yield_year.loc[avg_yield_year['Yield'].idxmax()]
print(highest_yield)


Crop             Coconut
Crop_Year           2009
Yield        9976.361986
Name: 221, dtype: object


In [12]:
df['State'] = df['State'].str.lower().str.strip()
df['District'] = df['District'].str.lower().str.strip()


In [13]:
avg_yield = df.groupby(['District', 'Crop'])['Yield'].transform('mean')
df['Production'] = df['Production'].fillna(df['Area'] * avg_yield)


In [14]:
odisha_wheat = df[(df['State'] == 'odisha') & (df['Crop'].str.lower() == 'wheat')]
print(odisha_wheat['Production'].sum())


121331.0


In [17]:
city_df = pd.read_csv("/city_day - city_day.csv")
merged = df.merge(city_df, left_on='District', right_on='City', how='inner')
print(merged.shape)


(0, 23)


In [18]:
city_df['Pollutant_ratio'] = city_df['SO2'] / city_df['NOx']
print(city_df[['City', 'SO2', 'NOx', 'Pollutant_ratio']])


                City    SO2    NOx  Pollutant_ratio
0          Ahmedabad  27.64  17.15         1.611662
1          Ahmedabad  24.55  16.46         1.491495
2          Ahmedabad  29.07  29.70         0.978788
3          Ahmedabad  18.59  17.97         1.034502
4          Ahmedabad  39.33  37.76         1.041578
...              ...    ...    ...              ...
29526  Visakhapatnam   8.55  19.54         0.437564
29527  Visakhapatnam  12.72  16.53         0.769510
29528  Visakhapatnam   8.42  18.33         0.459356
29529  Visakhapatnam   9.84  18.80         0.523404
29530  Visakhapatnam   2.10  14.05         0.149466

[29531 rows x 4 columns]


In [19]:
df['Calculated_Yield'] = df['Production'] / df['Area']
diff_sum = (df['Calculated_Yield'] - df['Yield']).sum()
print(diff_sum)


44295.80985101154


In [20]:
negatives = df[(df['Area'] < 0) | (df['Production'] < 0) | (df['Yield'] < 0)]
invalid_area = df[(df['Production'] > 0) & (df['Area'] == 0)]
future_years = df[df['Crop_Year'] > 2024]
print(negatives)
print(invalid_area)
print(future_years)


Empty DataFrame
Columns: [State, District, Crop, Crop_Year, Season, Area, Production, Yield, Calculated_Yield]
Index: []
Empty DataFrame
Columns: [State, District, Crop, Crop_Year, Season, Area, Production, Yield, Calculated_Yield]
Index: []
Empty DataFrame
Columns: [State, District, Crop, Crop_Year, Season, Area, Production, Yield, Calculated_Yield]
Index: []
